# Convert fore names to simplified (one word) fore names

In [1]:
import collections
import string
import pandas

import pubmedpy.names

In [2]:
name_df_paths = [
    'data/pubmed/authors.tsv.xz',
    'data/pmc/authors.tsv.xz',
    'data/ismb/keynotes.tsv',
]
path_to_df = {}
for path in name_df_paths:
    name_df = pandas.read_csv(path, keep_default_na=False, na_values=[''], sep='\t')
    path_to_df[path] = name_df

In [3]:
def simplify_names(dfs, name_column):
    names = pandas.concat(df[name_column] for df in dfs)
    simple_name_df = names.value_counts(dropna=True).reset_index()
    simple_name_df.columns = [name_column, 'n_authors']
    assert simple_name_df[name_column].isna().sum() == 0
    simplifier = getattr(pubmedpy.names, f'simplify_{name_column}')
    simple_name_df[f'{name_column}_simple'] = simple_name_df[name_column].map(lambda x: simplifier(x, lower=True))
    simple_name_df = simple_name_df.sort_values(name_column)
    return simple_name_df

In [4]:
simple_fore_name_df = simplify_names(path_to_df.values(), 'fore_name')
simple_fore_name_df

,fore_name,n_authors,fore_name_simple
27789,(Max) Zong-Ming,1,max
33502,", Yuxin",1,yuxin
57,A,237,None
4868,A A,8,None
21615,A Aldo,2,aldo
...,...,...,...
22218,Špela,2,špela
14198,Žiga,2,žiga
36806,Živa,1,živa
18749,Živadin,2,živadin


In [5]:
simple_last_name_df = simplify_names(path_to_df.values(), 'last_name')
simple_last_name_df

,last_name,n_authors,last_name_simple
36190,Rozowsky,1,rozowsky
33110,Slater,1,slater
38246,'t Hart,1,t hart
2282,'t Hoen,14,t hoen
35595,(Holly) Yang,1,holly) yang
...,...,...,...
17144,Žárský,2,žárský
37332,železný,1,železný
38478,žurauskienė,1,žurauskienė
24190,‘t Hoen,2,‘t hoen


In [6]:
simple_fore_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,fore_name,n_authors,fore_name_simple
2175,Siavash,17,siavash
3347,Ruchi,11,ruchi
5980,Nicholas V,6,nicholas
6095,Jonathan C,6,jonathan
11825,E A,3,None
12817,Tara L,3,tara
21419,Maria Anna,2,maria
14753,Sana,2,sana
24107,Tzu L,2,tzu
24074,Hulan,2,hulan


In [7]:
simple_last_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,last_name,n_authors,last_name_simple
492,Orengo,42,orengo
3609,Bortoluzzi,10,bortoluzzi
6884,van Eunen,6,van eunen
6951,Garcia-Martin,6,garcia-martin
8718,Blangiardo,5,blangiardo
11481,Grulke,4,grulke
11948,Tatusova,4,tatusova
11976,Tomaras,4,tomaras
12626,Dragan,4,dragan
14264,Kamath,3,kamath


In [8]:
# Most common last names
(
    simple_last_name_df
    .groupby('last_name_simple')
    .n_authors.sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(3)
)

,last_name_simple,n_authors
0,wang,3035
1,li,2566
2,zhang,2493


In [9]:
simple_fore_name_df.to_csv('data/names/fore-names.tsv.xz', sep='\t', index=False)
simple_last_name_df.to_csv('data/names/last-names.tsv.xz', sep='\t', index=False)